In [1]:
import re

# Patrón regex

In [27]:
EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.IGNORECASE)

PHONE_RE = re.compile(r"(\+?\d{1,3}[\s\-\.]?)?(\(?\d{2,4}\)?[\s\-\.]?)?\d{3}[\s\-\.]?\d{4}\b")

ADDRESS_RE = re.compile(r"""
(
    # --- US style: 135 Abbott St ---
    \b\d{1,6}\s+[A-Z0-9ÁÉÍÓÚÑa-záéíóúñ.\- ]{2,}\s+
    (St|Street|Ave|Avenue|Rd|Road|Blvd|Boulevard|Ln|Lane|Dr|Drive|Ct|Court|Pl|Place|Way|Pkwy|Parkway)\b
)
|
(
    # --- CO urban: Calle/Carrera/... 123 #45-67 (con letras y Sur/Este/Oeste) ---
    \b(?:calle|cl|carrera|cra|kr|avenida|av(?:\.|enida)?|transversal|trv|tv|diagonal|diag|dg|pasaje|psj)\s*
    \d{1,4}(?:ra|a|o)?[A-Za-z]{0,3}                   # 60N, 10A, 1ra, 25aa
    (?:\s*(?:norte|sur|este|oeste))?                  # opcional
    \s*(?:\#|n[°o\.]?|no\.?)\s*                        # separador (# / No / N°)
    \d{1,4}[A-Za-z]{0,3}                              # 45, 12C, 18B, 40f
    (?:\s*(?:norte|sur|este|oeste)\s*\d{0,4}[A-Za-z]{0,3})?  # sur67
    (?:\s*[-–]\s*\d{1,4}[A-Za-z]{0,3})?               # -67, -22
    \b
)
|
(
    # --- Rural / Km / Vía / Vereda ---
    \b(?:vereda)\s+[A-Za-zÁÉÍÓÚÑa-záéíóúñ ]+(?:,\s*parcela\s+[A-Za-zÁÉÍÓÚÑa-záéíóúñ0-9 ]+)?\b
)
|
(
    \b(?:kil[oó]metro|km)\s*\d+(?:[.,]\d+)?(?:\s*,?\s*(?:v[ií]a|via)\s+[A-Za-zÁÉÍÓÚÑa-záéíóúñ ]+)?\b
)
""", re.IGNORECASE | re.VERBOSE)

NAME_RE = re.compile(r"\b([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+){1,2})\b")

PUNT_RE = re.compile(r"[,.;:-]\b")

# Funciones

In [17]:
resp_vacia = "SIN_DESCRIPCION_TRAS_SANITIZACION"


def is_effectively_empty(s: str) -> bool:
    if s is None:
        return True
    return re.sub(r"[^A-Za-z0-9ÁÉÍÓÚÑáéíóúñ]+", "", str(s)) == ""

def cleanup_punctuation(t: str) -> str:
    t = re.sub(r"[,\.\-;:]{2,}", " ", t)

    t = re.sub(r"(?<=\s)[,\.\-;:]+(?=\s)", " ", t)

    t = t.strip(" ,.-;:")

    t = re.sub(r"\s+", " ", t).strip()
    return t

def sanitizar_texto(texto: str) -> str:

    if texto is None:
        return resp_vacia

    t = str(texto)

    # eliminar patrones claros
    t = EMAIL_RE.sub("", t)
    t = PHONE_RE.sub("", t)
    t = ADDRESS_RE.sub("", t)
    

    # Nombres propios (heurístico)
    t = NAME_RE.sub("", t)

    # Normalización
    t = re.sub(r"\s+", " ", t).strip()
    
    # Elimina puntuación
    t = cleanup_punctuation(t)

    return resp_vacia if is_effectively_empty(t) else t

In [18]:
tests = [
    # email + teléfono + nombre
    "Contactar a Juan Perez al +57 300-123-4567 o jperez@mail.com para validar.",
    # dirección + nombre
    "La propiedad está en 135 Abbott St y el responsable es Maria Lopez.",
    # caso que quede vacío
    "Juan Perez, +57 300-123-4567, jperez@mail.com, 22 Main Street"
]

res = [sanitizar_texto(txt) for txt in tests]

# Salida estructurada

In [23]:
nivel_alerta = {"BAJA", "MEDIA", "ALTA", "CRÍTICA"}

def salida_estructurada(request: dict) -> dict:
    
    if request["nivel_alerta"] not in nivel_alerta:
        raise ValueError(f"nivel_alerta inválido: {nivel_alerta}. Debe ser uno de {sorted(nivel_alerta)}")

    motivo_s = sanitizar_texto(request["motivo"])
    recomendacion_s = sanitizar_texto(request["recomendacion"])

    return {
        "id_propiedad": str(request["id_propiedad"]),
        "nivel_alerta": request["nivel_alerta"],
        "motivo_tecnico": motivo_s,
        "recomendacion": recomendacion_s,
    }


In [28]:
casos_prueba = [
    {
        "id_propiedad": "077-0044-0000",
        "nivel_alerta": "ALTA",
        "motivo": "Inconsistencia detectada. Llamar a Maria Lopez al (301) 555-0199 para confirmar datos.",
        "recomendacion": "Verificar en Calle 123 #45-67 y escribir a mlopez@ejemplo.com para soporte.",
    },
    {
        "id_propiedad": "030-0452-0000",
        "nivel_alerta": "MEDIA",
        "motivo": "Posible error de clasificación. Contactar a Carlos Perez al +57 310-555-0000.",
        "recomendacion": "Solicitar documentación adicional al correo carlos.perez@correo.co y ajustar registro si aplica.",
    },
    {
        "id_propiedad": "015-0467-0000",
        "nivel_alerta": "BAJA",
        "motivo": "Caso atípico por ubicación. Responsable: Ana Gomez, teléfono 3001234567.",
        "recomendacion": "Revisar historial de pagos y enviar notificación a ana.gomez@mail.com con pasos a seguir.",
    },
    {
        "id_propiedad":"077-0144-0000",
        "nivel_alerta": "CRÍTICA",
        "motivo":"Juan Perez, +57 300-123-4567, jperez@mail.com, Calle 123 #45-67",
        "recomendacion":"Calle 60N #12C-08"
    }
]

for caso in casos_prueba:
    print(salida_estructurada(caso))

{'id_propiedad': '077-0044-0000', 'nivel_alerta': 'ALTA', 'motivo_tecnico': 'Inconsistencia detectada. Llamar a al para confirmar datos', 'recomendacion': 'Verificar en y escribir a para soporte'}
{'id_propiedad': '030-0452-0000', 'nivel_alerta': 'MEDIA', 'motivo_tecnico': 'Posible error de clasificación. Contactar a al', 'recomendacion': 'Solicitar documentación adicional al correo y ajustar registro si aplica'}
{'id_propiedad': '015-0467-0000', 'nivel_alerta': 'BAJA', 'motivo_tecnico': 'Caso atípico por ubicación. Responsable: teléfono', 'recomendacion': 'Revisar historial de pagos y enviar notificación a con pasos a seguir'}
{'id_propiedad': '077-0144-0000', 'nivel_alerta': 'CRÍTICA', 'motivo_tecnico': 'SIN_DESCRIPCION_TRAS_SANITIZACION', 'recomendacion': 'SIN_DESCRIPCION_TRAS_SANITIZACION'}
